# **Homework 3 - Convolutional Neural Network**

This is the example code of homework 3 of the machine learning course by Prof. Hung-yi Lee.

In this homework, you are required to build a convolutional neural network for image classification, possibly with some advanced training tips.


There are three levels here:

**Easy**: Build a simple convolutional neural network as the baseline. (2 pts)

**Medium**: Design a better architecture or adopt different data augmentations to improve the performance. (2 pts)

**Hard**: Utilize provided unlabeled data to obtain better results. (2 pts)

## **About the Dataset**

The dataset used here is food-11, a collection of food images in 11 classes.

For the requirement in the homework, TAs slightly modified the data.
Please DO NOT access the original fully-labeled training data or testing labels.

Also, the modified dataset is for this course only, and any further distribution or commercial use is forbidden.

In [ ]:
#please download dataset to your drive and name it food-11_another.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q /content/drive/MyDrive/food-11_another.zip

## **Import Packages**

First, we need to import packages that will be used later.

In this homework, we highly rely on **torchvision**, a library of PyTorch.

In [ ]:
# Import necessary packages.
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder
import torchvision.models as models

# This is for the progress bar.
from tqdm.auto import tqdm
# from autoaugment import ImageNetPolicy

In [ ]:
# checkpoint = torch.load('./drive/MyDrive/final_train2')
# print(checkpoint['best_acc'])

tensor(0.7844, device='cuda:0')


## **Dataset, Data Loader, and Transforms**

Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Here, since our data are stored in folders by class labels, we can directly apply **torchvision.datasets.DatasetFolder** for wrapping data without much effort.

Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms.

In [ ]:
# It is important to do data augmentation in training.
# However, not every augmentation is useful.
# Please think about what kind of augmentation is helpful for food recognition.
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    # You may add some transforms here.
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])

# We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])


In [ ]:
# Batch size for training, validation, and testing.
# A greater batch size usually gives a more stable gradient.
# But the GPU memory is limited, so please adjust it carefully.
batch_size = 64
workers = 8

# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set = DatasetFolder("food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
unlabeled_set = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
test_set = DatasetFolder("food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

# Construct data loaders.
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## **Model**

The basic model here is simply a stack of convolutional layers followed by some fully-connected layers.

Since there are three channels for a color image (RGB), the input channels of the network must be three.
In each convolutional layer, typically the channels of inputs grow, while the height and width shrink (or remain unchanged, according to some hyperparameters like stride and padding).

Before fed into fully-connected layers, the feature map must be flattened into a single one-dimensional vector (for each image).
These features are then transformed by the fully-connected layers, and finally, we obtain the "logits" for each class.

### **WARNING -- You Must Know**
You are free to modify the model architecture here for further improvement.
However, if you want to use some well-known architectures such as ResNet50, please make sure **NOT** to load the pre-trained weights.
Using such pre-trained models is considered cheating and therefore you will be punished.
Similarly, it is your responsibility to make sure no pre-trained weights are used if you use **torch.hub** to load any modules.

For example, if you use ResNet-18 as your model:

model = torchvision.models.resnet18(pretrained=**False**) → This is fine.

model = torchvision.models.resnet18(pretrained=**True**)  → This is **NOT** allowed.

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # The arguments for commonly used modules:
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)

        # input image size: [3, 128, 128]
        # VGG19 https://github.com/Aleadinglight/Pytorch-VGG-19/blob/master/VGG_19.ipynb
        # batch norm added
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            ,nn.BatchNorm2d(64)
            ,nn.ReLU()
            ,nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            ,nn.BatchNorm2d(64)
            ,nn.ReLU()
            ,nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

            ,nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            ,nn.BatchNorm2d(128)
            ,nn.ReLU()
            ,nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            ,nn.BatchNorm2d(128)
            ,nn.ReLU()
            ,nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

            ,nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            ,nn.BatchNorm2d(256)
            ,nn.ReLU()
            ,nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            ,nn.BatchNorm2d(256)
            ,nn.ReLU()
            ,nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            ,nn.BatchNorm2d(256)
            ,nn.ReLU()
            #,nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            #,nn.BatchNorm2d(256)
            #,nn.ReLU()
            ,nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            #,nn.Dropout(0.5)

            ,nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            ,nn.BatchNorm2d(512)
            ,nn.ReLU()
            ,nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            ,nn.BatchNorm2d(512)
            ,nn.ReLU()
            ,nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            ,nn.BatchNorm2d(512)
            ,nn.ReLU()
            #,nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            #,nn.BatchNorm2d(512)
            #,nn.ReLU()
            ,nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            #,nn.Dropout(0.2)

            # ,nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            # ,nn.BatchNorm2d(512)
            # ,nn.ReLU()
            # ,nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            # ,nn.BatchNorm2d(512)
            # ,nn.ReLU()
            # ,nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            # ,nn.BatchNorm2d(512)
            # ,nn.ReLU()
            # ,nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            # ,nn.BatchNorm2d(512)
            # ,nn.ReLU()
            # ,nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

        )
        self.fc_layers = nn.Sequential(
            nn.Linear(512*8*8, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Dropout(p=0.6),
            nn.Linear(2048, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 11)
        )

    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x = self.cnn_layers(x)

        # The extracted feature map must be flatten before going to fully-connected layers.
        x = x.flatten(1)

        # The features are transformed by fully-connected layers to obtain the final logits.
        x = self.fc_layers(x)
        return x

## **Training**

You can finish supervised learning by simply running the provided code without any modification.

The function "get_pseudo_labels" is used for semi-supervised learning.
It is expected to get better performance if you use unlabeled data for semi-supervised learning.
However, you have to implement the function on your own and need to adjust several hyperparameters manually.

For more details about semi-supervised learning, please refer to [Prof. Lee's slides](https://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/semi%20(v3).pdf).

Again, please notice that utilizing external data (or pre-trained model) for training is **prohibited**.

In [ ]:
# this is for pseudo label
# copied from TIMIT dataset (previous homework)
from torch.utils.data import Dataset
import numpy as np

class pseudoDataset(Dataset):
    def __init__(self, X, y=None): 
      self.data = X
      if y is not None:
          self.label = y
      else:
          self.label = None
          print("something went wrong")
      #print(f"data result: {len(self.data) == len(self.label)}")

    def __getitem__(self, idx):
      return self.data[idx], self.label[idx]
        
    def __len__(self):
        return len(self.data)

In [ ]:
import gc
def get_pseudo_labels(dataset, model, threshold=0.8):
    # This functions generates pseudo-labels of a dataset using given model.
    # It returns an instance of DatasetFolder containing images whose prediction confidences exceed a given threshold.
    # You are NOT allowed to use any models trained on external data for pseudo-labeling.
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Construct a data loader.
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Make sure the model is in eval mode.
    model.eval()
    # Define softmax function.
    softmax = nn.Softmax(dim=-1)
    figures = []
    labels = []
    ### for debugging ###
    # print(f"type of dataset: {type(dataset)}")
    # print(f"type of img: {type(dataset[1][0])}")
    # print(f"shape of img: {dataset[1][0].shape}")
    # print(f"type of label: {type(dataset[1][1])}")
    # print(f"shape of label: {dataset[1][1].shape}")
    # Iterate over the dataset by batches.
    for batch in tqdm(data_loader):
        img, _ = batch

        # Forward the data
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(img.to(device))

        # Obtain the probability distributions by applying softmax on logits.
        probs = softmax(logits)
        # print(probs.shape) --> batch*11
        probs_cpu = probs.cpu()
        img_cpu = img.cpu()
        # ---------- TODO ----------
        # Filter the data and construct a new dataset.
        count = 0
        for row in probs_cpu:
          
          if (torch.max(row) > threshold):
            # print("row added")
            # print(row)
            # print(torch.max(row))
            _, val_pred = torch.max(row,0)
            # print(f"predicted type: {val_pred}")
            # print(f"type of pred: {type(val_pred)}")
            # print(f"type of my img: {type(img_cpu[count])}")
            # print(f"shape of my img: {img_cpu[count].shape}")
            figures.append(img_cpu[count])
            labels.append(val_pred.item())
          count += 1
        if count > batch_size:
          print("something wrong!!!!!!!")
    #figures.cpu()
    #labels.cpu()
    #pesudoset = pseudoDataset(figures, labels)
    # # Turn off the eval mode.
    model.train()
    del probs_cpu
    del img_cpu
    gc.collect()
    return pseudoDataset(figures, labels)

In [ ]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

### if you are swapping data, don't run this! ########
# Initialize a model, and put it on the device specified.
model = Classifier().to(device)
model.device = device

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)
##########################################################



In [ ]:

######## for training set and validation set swapping #######
# add_set, valid_set = torch.utils.data.random_split(valid_set, [30, 630])
# train_set= ConcatDataset([train_set, add_set])

# add_set, train_set = torch.utils.data.random_split(train_set, [30, 3080])
# valid_set= ConcatDataset([valid_set, add_set])

# valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
# train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)

####################################################################

############## restore from check point, code is modified from stackoverflow ########################
# model = Classifier().to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

# checkpoint = torch.load('./drive/MyDrive/final_train2') #change this for data swapping or continue training if disconnected from colab
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# # epoch = checkpoint['epoch']
# scheduler = checkpoint['scheduler']
# loss = checkpoint['loss']
# criterion = nn.CrossEntropyLoss()
# model.train()
########################################


scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=7,gamma=0.95)#這邊在slice時註解掉
# The number of training epochs.
n_epochs = 300 #change this to 30 for data swapping
model_path = './model1.ckpt'
# Whether to do semi-supervised learning.
valid_phase = False
do_semi = True
best_acc = -1
prev_acc = -1
# best1 = -1
# best2 = -1
# best3 = -1
first_change = 0
first_count = 0
first=0
state = 0
for epoch in range(n_epochs):
    # ---------- TODO ----------
    # In each epoch, relabel the unlabeled dataset for semi-supervised learning.
    # Then you can combine the labeled dataset and pseudo-labeled dataset for the training.

    # if : change pseudoset each 3 epoch. Also, due to the observation that sometime we miss the better model when mod 3, we add the latter case to do_semi after the validation accuracy had improved a lot
    if first != 0 and ((do_semi and epoch%3==2) or ((best_acc-prev_acc > (1/epoch)) and (best_acc > 0.5))): #and valid_phase == False  :
        # Obtain pseudo-labels for unlabeled data using trained model.
        
        
        pseudo_set = get_pseudo_labels(unlabeled_set, model)
        # if epoch > 120: #add test_set to data_set, but it didn't work so I don't use it
        #   pseudotest_set = get_pseudo_labels(test_set, model)   
        #   pseudo_set = ConcatDataset([pseudotest_set, pseudo_set]) 
        # Construct a new dataset and a data loader for training.
        # This is used in semi-supervised learning only.
        if pseudo_set==None:
          pass
        else:
          concat_dataset = ConcatDataset([train_set, pseudo_set])
          train_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
          # del pseudo_set
    #### auto dataset swapping, don't need to use it ####
    # if first != 0 and epoch >= 50 and epoch % 40 == 10:
    #   state = 1
    #   if best_acc > best1:
        
    #     torch.save(model.state_dict(), './drive/MyDrive/modelhw3_final_best1.ckpt')
    #     torch.save({
    #         'epoch': epoch,
    #         'model_state_dict': model.state_dict(),
    #         'optimizer_state_dict': optimizer.state_dict(),
    #         'loss': loss,
    #         'scheduler':scheduler,
    #         'best_acc': best_acc
    #         }, './drive/MyDrive/final_train_best1')
    #   first_change = 1
    #   model = best_model4
    #   add_set, valid_set = torch.utils.data.random_split(valid_set, [330, 330])
    #   train_set= ConcatDataset([train_set, add_set])
    #   valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
    #   train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
    #   valid_phase = True

    # elif first != 0 and epoch >= 50 and epoch % 50 == 20:
    #   state = 2
    #   if best_acc > best2:
    #     torch.save(model.state_dict(), './drive/MyDrive/modelhw3_final_best2.ckpt')
    #     torch.save({
    #         'epoch': epoch,
    #         'model_state_dict': model.state_dict(),
    #         'optimizer_state_dict': optimizer.state_dict(),
    #         'loss': loss,
    #         }, './drive/MyDrive/final_train_best2')
    #   first_change = 1
    #   model = best_model4
    #   add_set, valid_set = torch.utils.data.random_split(valid_set, [170, 160])
    #   train_set= ConcatDataset([train_set, add_set])
    #   valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
    #   train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
    #   valid_phase = True

    # elif first != 0 and epoch >= 50 and epoch % 40 == 30:
    #   state = 3
    #   if best_acc > best3:
    #     torch.save(model.state_dict(), './drive/MyDrive/modelhw3_final_best3.ckpt')
    #     torch.save({
    #         'epoch': epoch,
    #         'model_state_dict': model.state_dict(),
    #         'optimizer_state_dict': optimizer.state_dict(),
    #         'loss': loss,
    #         'scheduler':scheduler,
    #         'best_acc': best_acc
    #         }, './drive/MyDrive/final_train_best3')
    #   first_change = 1
    #   model = best_model4
    #   add_set, train_set = torch.utils.data.random_split(train_set, [330, len(train_set)-330])
    #   valid_set= ConcatDataset([valid_set, add_set])
    #   valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
    #   train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
    #   valid_phase = False
    ### 

    first = 1
    prev_acc = best_acc
    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    # Iterate the training set by batches.
    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #print(f"shape of imgs: {imgs.shape}")
        # print(f"imgs: {imgs}")
        #print(f"shape of labels: {labels.shape}")
        if labels.shape == torch.Size([1]):
          continue
        # print(f"labels: {labels}")
        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)

    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        # print("batch_size")
        imgs, labels = batch
        
        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
          logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)
    if (valid_acc > best_acc):
      if first_change == 0  :
        ### save checkpoint ###
        torch.save(model.state_dict(), model_path)
        torch.save(model.state_dict(), './drive/MyDrive/modelhw3_final2.ckpt')
        best_model4 = model
        torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': loss,
              'scheduler':scheduler,
              'best_acc': best_acc
              }, './drive/MyDrive/final_train2')
        print(f"save model with valid: {valid_acc}")
        best_acc = valid_acc
      # elif first_change < 4:
      #   first_change += 1
      # else:
      #   # torch.save(model.state_dict(), model_path)
      #   # torch.save(model.state_dict(), './drive/MyDrive/modelhw3_best1.ckpt')
      #   # best_model4 = model
      #   # torch.save({
      #   #       'epoch': epoch,
      #   #       'model_state_dict': model.state_dict(),
      #   #       'optimizer_state_dict': optimizer.state_dict(),
      #   #       'loss': loss,
      #   #       }, './drive/MyDrive/final_train1')
      #   torch.save(model.state_dict(), model_path)
      #   torch.save(model.state_dict(), f'./drive/MyDrive/modelhw3_best{state}.ckpt')
      #   best_model4 = model
      #   torch.save({
      #         'epoch': epoch,
      #         'model_state_dict': model.state_dict(),
      #         'optimizer_state_dict': optimizer.state_dict(),
      #         'loss': loss,
      #         'scheduler':scheduler,
      #         'best_acc': best_acc
      #         }, f'./drive/MyDrive/final_train_best{state}')
      #   print(f"save model with valid: {valid_acc}")
      #   best_acc = valid_acc
      ### save check point ###
      if epoch % 5 == 1: 
        torch.save(model.state_dict(), f'./drive/MyDrive/modelhw3_epoch{epoch}.ckpt')
        torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': loss,
              'scheduler':scheduler,
              'best_acc': best_acc
              }, f'./drive/MyDrive/final_train_epoch{epoch}')
      ###
    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
    scheduler.step()




/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



[ Train | 001/030 ] loss = 0.28008, acc = 0.91390



save model with valid: 0.7670454978942871
[ Valid | 001/030 ] loss = 0.94839, acc = 0.76705



[ Train | 002/030 ] loss = 0.16521, acc = 0.94729



save model with valid: 0.7713068127632141
[ Valid | 002/030 ] loss = 0.90966, acc = 0.77131



[ Train | 003/030 ] loss = 0.17333, acc = 0.94974



[ Valid | 003/030 ] loss = 0.98023, acc = 0.76080



[ Train | 004/030 ] loss = 0.15464, acc = 0.95417


In [ ]:
print(len(valid_set))
print(len(train_set))

660
3080


## **Testing**

For inference, we need to make sure the model is in eval mode, and the order of the dataset should not be shuffled ("shuffle=False" in test_loader).

Last but not least, don't forget to save the predictions into a single CSV file.
The format of CSV file should follow the rules mentioned in the slides.

### **WARNING -- Keep in Mind**

Cheating includes but not limited to:
1.   using testing labels,
2.   submitting results to previous Kaggle competitions,
3.   sharing predictions with others,
4.   copying codes from any creatures on Earth,
5.   asking other people to do it for you.

Any violations bring you punishments from getting a discount on the final grade to failing the course.

It is your responsibility to check whether your code violates the rules.
When citing codes from the Internet, you should know what these codes exactly do.
You will **NOT** be tolerated if you break the rule and claim you don't know what these codes do.


In [ ]:
# Make sure the model is in eval mode.
# Some modules like Dropout or BatchNorm affect if the model is in training mode.

model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in tqdm(test_loader):
    # A batch consists of image data and corresponding labels.
    # But here the variable "labels" is useless since we do not have the ground-truth.
    # If printing out the labels, you will find that it is always 0.
    # This is because the wrapper (DatasetFolder) returns images and labels for each batch,
    # so we have to create fake labels to make it work normally.
    imgs, labels = batch

    # We don't need gradient in testing, and we don't even have labels to compute loss.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        logits = model(imgs.to(device))

    # Take the class with greatest logit as prediction and record it.
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

In [ ]:
# Save predictions into the file.
with open('./predict.csv','w') as f:

    # The first row must be "Id, Category"
    f.write("Id,Category\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")